In [2]:
import sklearn
import numpy as np
import re
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import json
import os
import sys
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec

In [3]:
squad_dataset = json.load(open('SQuAD/train.json'))

In [4]:
def tokenize(sequence):
    # tokens = [token.replace("``", '"').replace("''", '"') for token in nltk.word_tokenize(sequence)]
    tokens = nltk.word_tokenize(sequence)
    return tokens


def token_map(context, context_tokens):
    word = ''
    current_token_id = 0
    token_map = dict()

    for cid, c in enumerate(context):
        if c != ' ':
            word += c
            context_token = context_tokens[current_token_id]
            if word == context_token:
                start = cid - len(word) + 1
                token_map[start] = [word, current_token_id]
                word = ''
                current_token_id += 1
    return token_map

In [13]:
def get_data(dataset):
    with open(os.path.join('./data/', 'train_context'), 'w') as context_file, \
    open(os.path.join('./data/', 'train_question'), 'w') as question_file, \
    open(os.path.join('./data/', 'train_answer'), 'w') as answer_file, \
    open(os.path.join('./data/', 'train_span'), 'w') as span_file:
        for data_id in range(len(dataset['data'])):
            data = dataset['data'][data_id]
            for para_id in range(len(data['paragraphs'])):
                para = data['paragraphs'][para_id]
                context = para['context']
                if context[0] == '\n':
                    context = context[-len(context)+1:]
                qas = para['qas']
                for qid in range(len(qas)):
                    question = qas[qid]['question']
                    answer_text = qas[qid]['answers'][0]['text']
                    answer_start = qas[qid]['answers'][0]['answer_start']
                    answer_end = answer_start + len(answer_text)
                    
                    question_file.write(question + '\n')
                    answer_file.write(answer_text + '\n')
                    span_file.write(str(answer_start) + ',' + str(answer_end) + '\n')
                
                context_file.write(context + '\n' + '-'*30 + '\n')
                question_file.write('-'*30 + '\n')
                answer_file.write('-'*30 + '\n')

In [ ]:
glove_input_file = './glove/glove.6B.300d.txt'
word2vec_output_file = './glove/glove.6B.300d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [6]:
word2vec_output_file = './glove/glove.6B.300d.txt.word2vec'
glove_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [86]:
def average_rep(x_train, q_data, model):
    x_avg = []
    q_avg = []
    avg = []
    for x in x_train:
        x_tokens = nltk.word_tokenize(x)
        x_vec = []
        for x_token in x_tokens:
            if x_token in model.wv.vocab:
                x_vec.append(model.wv[x_token])
        x_avg.append([a/len(x_vec) for a in sum(x_vec)])
    for ques in q_data:
        q_vec = []
        for q in ques:
            q_tokens = nltk.word_tokenize(q)
            for q_token in q_tokens:
                if q_token in model.wv.vocab:
                    q_vec.append(model.wv[q_token])
        q_avg.append([a/len(q_vec) for a in sum(q_vec)])
    for i in range(len(x_avg)):
        avg.append([(a+b)/2 for (a,b) in zip(x_avg[i], q_avg[i])])
    return avg

In [14]:
get_data(squad_dataset)

In [170]:
def remove_newlines(text):
    if text[0] == '\n':
        text = text[-len(text)+1:]
    elif text[-1] == '\n':
        text = text[:len(text)-1]
    return text

def get_cdata(context_file):
    c = context_file.read()
    lines = re.split('\n' + '-'*30 + '\n', c)[:-1]
    return lines

def get_qdata(xfile):
    final = []
    line = []
    for l in xfile:
        l = remove_newlines(l)
        if l != '-'*30:
            line.append(l)
        else:
            final.append(line)
            line = []
    return final

In [171]:
context_file = open('./data/train_context', 'r')
questions_file = open('./data/train_question', 'r')
context_data = get_cdata(context_file)
questions_data = get_qdata(questions_file)

In [172]:
glove_rep = average_rep(context_data[:2500], question_data[:2500], glove_model)

/home/rohithm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/home/rohithm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/home/rohithm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/rohithm/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [105]:
# Check lengths
tot = 0
for data_id in range(len(squad_dataset['data'])):
    data = squad_dataset['data'][data_id]
    for para_id in range(len(data['paragraphs'])):
        para = data['paragraphs'][para_id]
        tot += 1

print(tot)
print(c_tot)
print(len(context_data))
print(len(questions_data))

18896
19028
19028
18896
